In [40]:
from pprint import pprint
import requests 
import json
from datetime import date
import csv
import datetime
import pandas as pd
import time
from time import mktime
from datetime import datetime, timedelta
import numpy as np

In [32]:
#def getPushshiftData(query, after, before, sub):
def getPushshiftData(after, before, sub):
    #url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
#     url = 'https://api.pushshift.io/reddit/search/submission/?size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    
    url = 'https://api.pushshift.io/reddit/search/submission/?size=1000&after={}&before={}&subreddit={}'.format(after, before, sub)
    
    #url = 'https://api.pushshift.io/reddit/search/submission/?link_flair_text=Resources&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    #SEE ABOUT SOME OTHER THING FOR FLAIR doesn't seem to be working
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
#     print(data)
    return data['data']

In [33]:
# def collectSubData(subm):
#     subData = list() #list to store data points
#     title = subm['title']
#     url = subm['url']
#     try:
#         flair = subm['link_flair_text']
#     except KeyError:
#         flair = "NaN"  
#     try:
#         flair_acss = subm['author_flair_css_class']
#     except KeyError:
#         flair_acss = "NaN"
#     try:
#         flair_atext = subm['author_flair_text']
#     except KeyError:
#         flair_atext = "NaN"
#     try:
#         flair_lcss = subm['link_flair_css_class']
#     except KeyError:
#         flair_lcss = "NaN"
#     author = subm['author']
#     sub_id = subm['id']
#     score = subm['score']
#     created = datetime.fromtimestamp(subm['created_utc']) #1520561700.0
#     numComms = subm['num_comments']
#     permalink = subm['permalink']
    
    #instead of appending all to subData like this, have to change it so each data type appends to its own thing we can 
    #just use as a column. Then you can put them all in a list named data if you want
    #but theeeen we'd have to pass every list into this and return all of them
#     subData.append((sub_id,title,url,author,score,created,numComms,permalink,flair,flair_acss,flair_atext,flair_lcss))
#     subStats[sub_id] = subData
    
    #print(subStats)

In [44]:
def collectSubData(subm):
    pprint(subm)
#     id.append(subm['id'])
    title.append(subm['title'])
#     url.append(subm['url'])
    try:
        flair.append(subm['link_flair_text'])
    except KeyError:
        flair.append("NaN")
    author.append(subm['author'])
    created.append(datetime.fromtimestamp(subm['created_utc'])) #1520561700.0
    numComms.append(subm['num_comments'])
    permalink.append(subm['permalink'])
    try:
        selftext.append(subm['selftext'])
    except KeyError:
        selftext.append("NaN")
    
#     print(subm['selftext'])


In [45]:
sub='COVID19_support'

before = "2020-05-15" 
after = "2020-05-14"  
#query = "Screenshot"
# subCount = 0
# subStats = {}

#add whatever data you want below, adjust range to match number of columns and give them their title in subredditdata
# id, title, url, author, created, numComms, permalink, flair = ([] for i in range(8))
# subredditdata = {"sub id": id, "title": title, "url": url, "author": author, "created": created, "numComms": numComms,
#                  "permalink": permalink, "flair": flair}

title, selftext, permalink, author, created = ([] for i in range(5)) #find out the parameter for the actual post text
subredditdata = {"author username": author, "title": title, "text body": selftext, "date created": created, "permalink": permalink}

In [46]:
#data = getPushshiftData(query, after, before, sub)# Will run until all posts have been gathered 
data = getPushshiftData(after, before, sub)

# from the 'after' date up until before date
while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = getPushshiftData(after, before, sub)
    #data = getPushshiftData(query, after, before, sub)
    
print(len(data))

https://api.pushshift.io/reddit/search/submission/?size=1000&after=2020-05-14&before=2020-05-15&subreddit=COVID19_support
{'all_awardings': [],
 'allow_live_comments': False,
 'author': 'OTOT4',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_5m6i5vdh',
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1589415782,
 'domain': 'self.COVID19_support',
 'full_link': 'https://www.reddit.com/r/COVID19_support/comments/gjbfr3/exposure_therapy_for_integrating_back_into_the/',
 'gildings': {},
 'id': 'gjbfr3',
 'is_crosspostable': True,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': False,
 'is_robot_indexable': True,
 'is_self': True,
 'is_video': False,
 'link_flair_background_color': '',
 'link_flair_richtext': [],
 'link_flair_text_color': 'dark',
 'link_flair_type': 'text',
 'l

NameError: name 'flair' is not defined

In [291]:
# print(str(len(subStats)) + " submissions have added to list")
# print("1st entry is:")
# print(list(subStats.values())[0][0][1] + " created: " + str(list(subStats.values())[0][0][5]))
# print("Last entry is:")
# print(list(subStats.values())[-1][0][1] + " created: " + str(list(subStats.values())[-1][0][5]))

In [292]:
# def updateSubs_file():
#     upload_count = 0
#     #location = "\\Reddit Data\\"
    
#     #print("input filename of submission file, please add .csv")
#     filename = "redditdata.csv"#input()
#     #file = location + filename
#     file = filename
#     with open(file, 'w', newline='', encoding='utf-8') as file: 
#     # with open(file, 'w', newline='', encoding='utf-8') as file: 
#         a = csv.writer(file, delimiter=',')
#         #headers = ["Post ID","Title","Url","Author","Score","Publish Date","Total No. of Comments","Permalink","Flair"]
#         headers = ["Post ID","Title","Url","Author","Score","Publish Date","Total No. of Comments","Permalink","Flair","flair_acss","flair_atext","flair_lcss"]
#         a.writerow(headers)
#         for sub in subStats:
#             a.writerow(subStats[sub][0])
#             upload_count+=1
            
#         print(str(upload_count) + " submissions have been uploaded") #updateSubs_file()

In [293]:
# updateSubs_file()

In [294]:
#subStats['f43bg7'][0][0]
# subStats['f43bg7'][0][0]

In [295]:
# id = []
# title = []
# id, title = ([] for i in range(2))
# for sub in subStats:
# #     print(sub)
#     id.append(subStats[sub][0][0])
#     title.append(subStats[sub][0][1])

# data = {'post id': id, 'title': title, }

# print(subStats[sub][0])


In [296]:
redditdata = pd.DataFrame(subredditdata)
#redditdata.reset_index(drop = True)
redditdata.head()
# print(subredditdata)


,author username,title,text body,date created,permalink
0,OTOT4,Exposure therapy for integrating back into the...,Hey all! I came across this page by accident a...,2020-05-13 20:23:02,/r/COVID19_support/comments/gjbfr3/exposure_th...
1,lillybee1234,"I did it, I went into a store",I posted a few days ago about how I couldn't e...,2020-05-13 20:56:59,/r/COVID19_support/comments/gjc0qb/i_did_it_i_...
2,PlayfulTitan18,Hey everyone I thought I would give an update ...,So I made a post a while back about how this p...,2020-05-13 21:34:11,/r/COVID19_support/comments/gjcmkn/hey_everyon...
3,Old_boy7,Anyone else have upset stomach as the primary ...,I should preference I never got tested. About ...,2020-05-13 21:45:50,/r/COVID19_support/comments/gjctdw/anyone_else...
4,rendervelvet,Navigating Extended Family and Social Distanci...,I (mid 30s) am currently sharing a household w...,2020-05-13 22:24:25,/r/COVID19_support/comments/gjdfb9/navigating_...


In [298]:
# if text body - blank or [removed], drop row
# redditdata = redditdata.query(redditdata['text body'] == "")
# redditdata = redditdata[redditdata['text body'] != ("" || "[removed]")
redditdata = redditdata[redditdata['text body'] != "[removed]"]
redditdata = redditdata[redditdata['text body'] != ""]
redditdata

TypeError: unsupported operand type(s) for |: 'str' and 'str'

In [ ]:
redditdata.to_csv("test.csv")